In [ ]:
!pip install datasets
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
!pip install -U lightrag[ollama]
!pip install rouge-score

In [ ]:
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [ ]:
from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

In [ ]:
!ollama create myLLM -f /kaggle/input/modelfile/Modelfile

In [ ]:
!ollama list

In [ ]:
from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient

import time


qa_template = r"""<SYS>
You are a helpful assistant. Just answer the question and avoid chatting or explaining the reason for your output. You are specilized in numerical reasoning. Below a full example is provided, but I briefly explain a syntax for you: If your calculations end up as (5 / 10) * 100 + 23 - 888 you must give me this notation instead without any exlanation: divide(5, 10), multiply(#0, 100), add(#1, 23), subtract(#2, 88). The #N notation represents the expression in step N starting from 0 and #2 means the outcome until the last part of step 2 (containing 0, 1, and 2). All the operations we use: add(12, 32) = 12 + 32, subtract(2, 56) = 2 - 56, multiply(123, 3) = 123 * 3, divide(56, 23) = 56 ÷ 23, greater(2, 9) = 9, table_max, exp(2, 3) = 2 ^ 3 = 8

example1: post_text: ['is used to monitor the risk in the loan classes.', 'loans with higher fico scores and lower ltvs tend to have a lower level of risk.', 'conversely, loans with lower fico scores, higher ltvs, and in certain geographic locations tend to have a higher level of risk.', 'in the first quarter of 2013, we refined our process for the home equity and residential real estate asset quality indicators shown in the following tables.', 'these refinements include, but are not limited to, improvements in the process for determining lien position and ltv in both table 67 and table 68.', 'additionally, as of the first quarter of 2013, we are now presenting table 67 at recorded investment as opposed to our prior presentation of outstanding balance.', 'table 68 continues to be presented at outstanding balance.', 'both the 2013 and 2012 period end balance disclosures are presented in the below tables using this refined process.', 'consumer purchased impaired loan class estimates of the expected cash flows primarily determine the credit impacts of consumer purchased impaired loans.', 'consumer cash flow estimates are influenced by a number of credit related items, which include, but are not limited to : estimated real estate values, payment patterns, updated fico scores, the current economic environment, updated ltv ratios and the date of origination.', 'these key factors are monitored to help ensure that concentrations of risk are mitigated and cash flows are maximized.','see note 6 purchased loans for additional information.', 'table 66 : home equity and residential real estate balances in millions december 31 december 31 home equity and residential real estate loans 2013 excluding purchased impaired loans ( a ) $ 44376 $ 42725 home equity and residential real estate loans 2013 purchased impaired loans ( b ) 5548 6638 government insured or guaranteed residential real estate mortgages ( a ) 1704 2279 purchase accounting adjustments 2013 purchased impaired loans ( 116 ) ( 482 ) total home equity and residential real estate loans ( a ) $ 51512 $ 51160 ( a ) represents recorded investment.', '( b ) represents outstanding balance.', '136 the pnc financial services group, inc.', '2013 form 10-k.'] table: [['in millions', 'december 31 2013', 'december 31 2012'], ['home equity and residential real estate loans 2013 excluding purchased impaired loans ( a )', '$ 44376', '$ 42725'], ['home equity and residential real estate loans 2013 purchased impaired loans ( b )', '5548', '6638'], ['government insured or guaranteed residential real estate mortgages ( a )', '1704', '2279'], ['purchase accounting adjustments 2013 purchased impaired loans', '-116 ( 116 )', '-482 ( 482 )'], ['total home equity and residential real estate loans ( a )', '$ 51512', '$ 51160']] pre_text: ['is used to monitor the risk in the loan classes.', 'loans with higher fico scores and lower ltvs tend to have a lower level of risk.', 'conversely, loans with lower fico scores, higher ltvs, and in certain geographic locations tend to have a higher level of risk.', 'in the first quarter of 2013, we refined our process for the home equity and residential real estate asset quality indicators shown in the following tables.', 'these refinements include, but are not limited to, improvements in the process for determining lien position and ltv in both table 67 and table 68.', 'additionally, as of the first quarter of 2013, we are now presenting table 67 at recorded investment as opposed to our prior presentation of outstanding balance.', 'table 68 continues to be presented at outstanding balance.', 'both the 2013 and 2012 period end balance disclosures are presented in the below tables using this refined process.', 'consumer purchased impaired loan class estimates of the expected cash flows primarily determine the credit impacts of consumer purchased impaired loans.', 'consumer cash flow estimates are influenced by a number of credit related items, which include, but are not limited to : estimated real estate values, payment patterns, updated fico scores, the current economic environment, updated ltv ratios and the date of origination.', 'these key factors are monitored to help ensure that concentrations of risk are mitigated and cash flows are maximized.','see note 6 purchased loans for additional information.', 'table 66 : home equity and residential real estate balances in millions december 31 december 31 home equity and residential real estate loans 2013 excluding purchased impaired loans ( a ) $ 44376 $ 42725 home equity and residential real estate loans 2013 purchased impaired loans ( b ) 5548 6638 government insured or guaranteed residential real estate mortgages ( a ) 1704 2279 purchase accounting adjustments 2013 purchased impaired loans ( 116 ) ( 482 ) total home equity and residential real estate loans ( a ) $ 51512 $ 51160 ( a ) represents recorded investment.', '( b ) represents outstanding balance.', '136 the pnc financial services group, inc.', '2013 form 10-k.'] Question: was december 31 2013 home equity and residential real estate loans 2013 excluding purchased impaired loans greater than purchased impaired loans?
Do not explain when there is missing data or the question is not relavant. just output this: "Not Found!".
answer1: greater(44376, 5548)




example2: post_text: ['.'] table: [['2017', '$ 27000'], ['2018', '27000'], ['2019', '63000'], ['2020', '25000'], ['2021', '86250'], ['2022 and thereafter', '600000'], ['total scheduled maturities of long term debt', '$ 828250'], ['current maturities of long term debt', '$ 27000']] pre_text: ['be adjusted by reference to a grid ( the 201cpricing grid 201d ) based on the consolidated leverage ratio and ranges between 1.00% ( 1.00 % ) to 1.25% ( 1.25 % ) for adjusted libor loans and 0.00% ( 0.00 % ) to 0.25% ( 0.25 % ) for alternate base rate loans.', 'the weighted average interest rate under the outstanding term loans and revolving credit facility borrowings was 1.6% ( 1.6 % ) and 1.3% ( 1.3 % ) during the years ended december 31, 2016 and 2015, respectively.', 'the company pays a commitment fee on the average daily unused amount of the revolving credit facility and certain fees with respect to letters of credit.', 'as of december 31, 2016, the commitment fee was 15.0 basis points.','since inception, the company incurred and deferred $ 3.9 million in financing costs in connection with the credit agreement.', '3.250% ( 3.250 % ) senior notes in june 2016, the company issued $ 600.0 million aggregate principal amount of 3.250% ( 3.250 % ) senior unsecured notes due june 15, 2026 ( the 201cnotes 201d ).', 'the proceeds were used to pay down amounts outstanding under the revolving credit facility.', 'interest is payable semi-annually on june 15 and december 15 beginning december 15, 2016.', 'prior to march 15, 2026 ( three months prior to the maturity date of the notes ), the company may redeem some or all of the notes at any time or from time to time at a redemption price equal to the greater of 100% ( 100 % ) of the principal amount of the notes to be redeemed or a 201cmake-whole 201d amount applicable to such notes as described in the indenture governing the notes, plus accrued and unpaid interest to, but excluding, the redemption date.', 'on or after march 15, 2026 ( three months prior to the maturity date of the notes ), the company may redeem some or all of the notes at any time or from time to time at a redemption price equal to 100% ( 100 % ) of the principal amount of the notes to be redeemed, plus accrued and unpaid interest to, but excluding, the redemption date.', 'the indenture governing the notes contains covenants, including limitations that restrict the company 2019s ability and the ability of certain of its subsidiaries to create or incur secured indebtedness and enter into sale and leaseback transactions and the company 2019s ability to consolidate, merge or transfer all or substantially all of its properties or assets to another person, in each case subject to material exceptions described in the indenture.', 'the company incurred and deferred $ 5.3 million in financing costs in connection with the notes.', 'other long term debt in december 2012, the company entered into a $ 50.0 million recourse loan collateralized by the land, buildings and tenant improvements comprising the company 2019s corporate headquarters.', 'the loan has a seven year term and maturity date of december 2019.', 'the loan bears interest at one month libor plus a margin of 1.50% ( 1.50 % ), and allows for prepayment without penalty.', 'the loan includes covenants and events of default substantially consistent with the company 2019s credit agreement discussed above.', 'the loan also requires prior approval of the lender for certain matters related to the property, including transfers of any interest in the property.', 'as of december 31, 2016 and 2015, the outstanding balance on the loan was $ 42.0 million and $ 44.0 million, respectively.', 'the weighted average interest rate on the loan was 2.0% ( 2.0 % ) and 1.7% ( 1.7 % ) for the years ended december 31, 2016 and 2015, respectively.', 'the following are the scheduled maturities of long term debt as of december 31, 2016 : ( in thousands ).'] Question: what is the interest expense based on the average outstanding loan balance in 2016?
answer2: add(42.0, 44.0), divide(#0, const_3), multiply(#1, 2.0%), multiply(#2, const_100000)


example3: post_text: ['differences arising from actual experience or changes in assumptions might materially affect retirement related benefit plan obligations and the funded status.', 'actuarial gains and losses arising from differences from actual experience or changes in assumptions are deferred in accumulated other comprehensive income.', 'this unrecognized amount is amortized to the extent it exceeds 10% ( 10 % ) of the greater of the plan 2019s benefit obligation or plan assets.', 'the amortization period for actuarial gains and losses is the estimated average remaining service life of the plan participants, which is approximately 10 years.', 'cas expense 2014in addition to providing the methodology for calculating retirement related benefit plan costs, cas also prescribes the method for assigning those costs to specific periods.', 'while the ultimate liability for such costs under fas and cas is similar, the pattern of cost recognition is different.', 'the key drivers of cas pension expense include the funded status and the method used to calculate cas reimbursement for each of our plans as well as our expected long-term rate of return on assets assumption.', 'unlike fas, cas requires the discount rate to be consistent with the expected long-term rate of return on assets assumption, which changes infrequently given its long-term nature.', 'as a result, changes in bond or other interest rates generally do not impact cas.', 'in addition, unlike under fas, we can only allocate pension costs for a plan under cas until such plan is fully funded as determined under erisa requirements.', 'other fas and cas considerations 2014we update our estimates of future fas and cas costs at least annually based on factors such as calendar year actual plan asset returns, final census data from the end of the prior year, and other actual and projected experience.', 'a key driver of the difference between fas and cas expense ( and consequently, the fas/cas adjustment ) is the pattern of earnings and expense recognition for gains and losses that arise when our asset and liability experiences differ from our assumptions under each set of requirements.', 'under fas, our net gains and losses exceeding the 10% ( 10 % ) corridor are amortized.'] table: [['( $ in millions )', 'increase ( decrease ) in 2012 expense', 'increase ( decrease ) in december 31 2011 obligations'], ['25 basis point decrease in discount rate', '$ 18', '$ 146'], ['25 basis point increase in discount rate', '-17 ( 17 )', '-154 ( 154 )'], ['25 basis point decrease in expected return on assets', '8', 'n.a.'], ['25 basis point increase in expected return on assets', '-8 ( 8 )', 'n.a.']] pre_text: ['discount rate 2014the assumed discount rate is used to determine the current retirement related benefit plan expense and obligations, and represents the interest rate that is used to determine the present value of future cash flows currently expected to be required to effectively settle a plan 2019s benefit obligations.', 'the discount rate assumption is determined for each plan by constructing a portfolio of high quality bonds with cash flows that match the estimated outflows for future benefit payments to determine a single equivalent discount rate.', 'benefit payments are not only contingent on the terms of a plan, but also on the underlying participant demographics, including current age, and assumed mortality.', 'we use only bonds that are denominated in u.s.', 'dollars, rated aa or better by two of three nationally recognized statistical rating agencies, have a minimum outstanding issue of $ 50 million as of the measurement date, and are not callable, convertible, or index linked.','since bond yields are generally unavailable beyond 30 years, we assume those rates will remain constant beyond that point.', 'taking into consideration the factors noted above, our weighted average discount rate for pensions was 5.23% ( 5.23 % ) and 5.84% ( 5.84 % ), as of december 31, 2011 and 2010, respectively.', 'our weighted average discount rate for other postretirement benefits was 4.94% ( 4.94 % ) and 5.58% ( 5.58 % ) as of december 31, 2011 and 2010, respectively.', 'expected long-term rate of return 2014the expected long-term rate of return on assets is used to calculate net periodic expense, and is based on such factors as historical returns, targeted asset allocations, investment policy, duration, expected future long-term performance of individual asset classes, inflation trends, portfolio volatility, and risk management strategies.', 'while studies are helpful in understanding current trends and performance, the assumption is based more on longer term and prospective views.', 'in order to reflect expected lower future market returns, we have reduced the expected long-term rate of return assumption from 8.50% ( 8.50 % ), used to record 2011 expense, to 8.00% ( 8.00 % ) for 2012.', 'the decrease in the expected return on assets assumption is primarily related to lower bond yields and updated return assumptions for equities.', 'unless plan assets and benefit obligations are subject to remeasurement during the year, the expected return on pension assets is based on the fair value of plan assets at the beginning of the year.', 'an increase or decrease of 25 basis points in the discount rate and the expected long-term rate of return assumptions would have had the following approximate impacts on pensions : ( $ in millions ) increase ( decrease ) in 2012 expense increase ( decrease ) in december 31, 2011 obligations.'] Question: what was the net change in the 25 basis point decrease and increase in discount rate in 2012 in millions
answer3: add(18, -17)

example4: post_text: ['we use ebitda, as defined by our financial covenants, as a non-gaap measure.', 'this supplemental information related to ebitda represents a measure not in accordance with u.s.', 'gaap and should be viewed in addition to, not instead of, our consolidated financial statements and notes thereto.','shelf registration statement on september 3, 2015, we filed a shelf registration statement with the sec, registering the offer and sale from time to time of an indeterminate amount of, among other securities, debt securities, preference shares, class a ordinary shares and convertible securities.', 'our ability to access the market as a source of liquidity is dependent on investor demand, market conditions and other factors..'] table: [['years ended december 31,', '2015', '2014'], ['net income', '1422', '1431'], ['interest expense', '273', '255'], ['income taxes', '267', '334'], ['depreciation of fixed assets', '229', '242'], ['amortization of intangible assets', '314', '352'], ['total ebitda', '2505', '2614'], ['total debt', '5737', '5582'], ['total debt-to-ebitda ratio', '2.3', '2.1']] pre_text: ['on may 20, 2015, aon plc issued $ 600 million of 4.750% ( 4.750 % ) senior notes due may 2045.', 'the 4.750% ( 4.750 % ) notes due may 2045 are fully and unconditionally guaranteed by aon corporation.', 'we used the proceeds of the issuance for general corporate purposes.', 'on september 30, 2015, $ 600 million of 3.50% ( 3.50 % ) senior notes issued by aon corporation matured and were repaid.', 'on november 13, 2015, aon plc issued $ 400 million of 2.80% ( 2.80 % ) senior notes due march 2021.', 'the 2.80% ( 2.80 % ) notes due march 2021 are fully and unconditionally guaranteed by aon corporation.', 'we used the proceeds of the issuance for general corporate purposes.', 'credit facilities as of december 31, 2015, we had two committed credit facilities outstanding : our $ 400 million u.s.', 'credit facility expiring in march 2017 ( the "2017 facility" ) and $ 900 million multi-currency u.s.', 'credit facility expiring in february 2020 ( the "2020 facility" ).', 'the 2020 facility was entered into on february 2, 2015 and replaced the previous 20ac650 million european credit facility.', 'each of these facilities is intended to support our commercial paper obligations and our general working capital needs.', 'in addition, each of these facilities includes customary representations, warranties and covenants, including financial covenants that require us to maintain specified ratios of adjusted consolidated ebitda to consolidated interest expense and consolidated debt to adjusted consolidated ebitda, tested quarterly.', 'at december 31, 2015, we did not have borrowings under either the 2017 facility or the 2020 facility, and we were in compliance with the financial covenants and all other covenants contained therein during the twelve months ended december 31, 2015.', 'effective february 2, 2016, the 2020 facility terms were extended for 1 year and will expire in february 2021 our total debt-to-ebitda ratio at december 31, 2015 and 2014, is calculated as follows:.'] Question: what is the ebit of aon for 2015?
answer4: subtract(2505, 314), subtract(#0, 229)

example5: post_text: ['oil sands mining segment we hold a 20 percent non-operated interest in the aosp, an oil sands mining and upgrading joint venture located in alberta, canada.', 'the joint venture produces bitumen from oil sands deposits in the athabasca region utilizing mining techniques and upgrades the bitumen to synthetic crude oils and vacuum gas oil.', 'the aosp 2019s mining and extraction assets are located near fort mcmurray, alberta, and include the muskeg river and the jackpine mines.', 'gross design capacity of the combined mines is 255000 ( 51000 net to our interest ) barrels of bitumen per day.', 'the aosp operations use established processes to mine oil sands deposits from an open-pit mine, extract the bitumen and upgrade it into synthetic crude oils.', 'ore is mined using traditional truck and shovel mining techniques.', 'the mined ore passes through primary crushers to reduce the ore chunks in size and is then sent to rotary breakers where the ore chunks are further reduced to smaller particles.', 'the particles are combined with hot water to create slurry.', 'the slurry moves through the extraction process where it separates into sand, clay and bitumen-rich froth.', 'a solvent is added to the bitumen froth to separate out the remaining solids, water and heavy asphaltenes.', 'the solvent washes the sand and produces clean bitumen that is required for the upgrader to run efficiently.', 'the process yields a mixture of solvent and bitumen which is then transported from the mine to the scotford upgrader via the approximately 300-mile corridor pipeline.', "the aosp's scotford upgrader is located at fort saskatchewan, northeast of edmonton, alberta.", 'the bitumen is upgraded at scotford using both hydrotreating and hydroconversion processes to remove sulfur and break the heavy bitumen molecules into lighter products.', 'blendstocks acquired from outside sources are utilized in the production of our saleable products.', 'the upgrader produces synthetic crude oils and vacuum gas oil.', 'the vacuum gas oil is sold to an affiliate of the operator under a long-term contract at market-related prices, and the other products are sold in the marketplace.', 'as of december 31, 2014, we own or have rights to participate in developed and undeveloped leases totaling approximately 163000 gross ( 33000 net ) acres.', 'the underlying developed leases are held for the duration of the project, with royalties payable to the province of alberta.','synthetic crude oil sales volumes for 2014 averaged 50 mbbld and net-of-royalty production was 41 mbbld.', 'in december 2013, a jackpine mine expansion project received conditional approval from the canadian government.', 'the project includes additional mining areas, associated processing facilities and infrastructure.', 'the government conditions relate to wildlife, the environment and aboriginal health issues.', 'we will evaluate the potential expansion project and government conditions after infrastructure reliability initiatives are completed.', 'the governments of alberta and canada have agreed to partially fund quest ccs for $ 865 million canadian.', 'in the third quarter of 2012, the energy and resources conservation board ( "ercb" ), alberta\'s primary energy regulator at that time, conditionally approved the project and the aosp partners approved proceeding to construct and operate quest ccs.', 'government funding commenced in 2012 and continued as milestones were achieved during the development, construction and operating phases.', 'failure of the aosp to meet certain timing, performance and operating objectives may result in repaying some of the government funding.', 'construction and commissioning of quest ccs is expected to be completed by late 2015..'] table: [['( in thousands )', 'net undeveloped acres expiring year ended december 31, 2015', 'net undeveloped acres expiring year ended december 31, 2016', 'net undeveloped acres expiring year ended december 31, 2017'], ['u.s.', '211', '150', '94'], ['e.g.', '36', '2014', '2014'], ['other africa', '1950', '1502', '1089'], ['total africa', '1986', '1502', '1089'], ['other international', '88', '2014', '2014'], ['total', '2285', '1652', '1183']] pre_text: ['in the ordinary course of business, based on our evaluations of certain geologic trends and prospective economics, we have allowed certain lease acreage to expire and may allow additional acreage to expire in the future.', 'if production is not established or we take no other action to extend the terms of the leases, licenses, or concessions, undeveloped acreage listed in the table below will expire over the next three years.', 'we plan to continue the terms of many of these licenses and concession areas or retain leases through operational or administrative actions.', 'net undeveloped acres expiring year ended december 31.'] Question: what percentage of net undeveloped acres are located in the u.s in 2015?
answer5: divide(211, 2285)

</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})
        

In [ ]:
from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display



model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "myLLM"}
}
qa = SimpleQA(**model)
output=qa("what is happiness")
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
prompt = """
other items on our consolidated financial statements have been appropriately adjusted from the amounts provided in the earnings release , including a reduction of our full year 2016 gross profit and income from operations by $ 2.9 million , and a reduction of net income by $ 1.7 million. .
( in thousands )	at december 31 , 2016	at december 31 , 2015	at december 31 , 2014	at december 31 , 2013	at december 31 , 2012
cash and cash equivalents	$ 250470	$ 129852	$ 593175	$ 347489	$ 341841
working capital ( 1 )	1279337	1019953	1127772	702181	651370
inventories	917491	783031	536714	469006	319286
total assets	3644331	2865970	2092428	1576369	1155052
total debt including current maturities	817388	666070	281546	151551	59858
total stockholders 2019 equity	$ 2030900	$ 1668222	$ 1350300	$ 1053354	$ 816922
( 1 ) working capital is defined as current assets minus current liabilities. Question: what was the percentage change in inventories from 2015 to 2016?

"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
prompt = """
convert from infix notation into postfix notation: 8 * (11+43) ÷ 67
"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
prompt = """
(12 * 67) ^ (23) - 152
"""
output=qa(prompt)
display(Markdown(f"**Answer:** {output.data}"))

In [ ]:
print("The true answer for above:")

In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

generated = "Subtract(12.393, 22.366), divide(#0, 22.366)"
target = "subtract(22.366, 12.393), divide(#0, 12.393)"



scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

# Calculate ROUGE scores
scores = scorer.score(target, generated)

print("rouge1: ", scores['rouge1'])
print("rouge2: ", scores['rouge2'])
print("rougeL: ", scores['rougeL'])


# BLEU score calculation
bleu_score = sentence_bleu([target], generated)

print(f"BLEU score: {bleu_score}")

In [ ]:
from datasets import load_dataset

# Load FinQA dataset
dataset = load_dataset("ibm/finqa", trust_remote_code=True)

# Check the structure
print(dataset)
print(dataset['train'][0])  # View the first training example

In [ ]:

dataset['train'][10]['program_re']

In [ ]:
dataset['train'][10]['question']

In [ ]:
from transformers import AutoTokenizer

# Authentication token
token = "Costume Tokens To download the llama model (The user must apply for the access)"

# Initialize tokenizer for Llama 3.1
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", token=token)

# Add padding token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Preprocessing function with corrected target handling
def preprocess(example):
    # Safeguard for missing fields
    post_text = example.get('post_text', '')
    pre_text = example.get('pre_text', '')
    table = example.get('table', '')
    question = example.get('question', '')
    program = example.get('program_re', '')

    # Format input and target text
    input_text = f"post_text: {post_text} table: {table} pre_text: {pre_text} Question: {question}"
    target_text = program  

    # Tokenize input and target text
    inputs = tokenizer(input_text, truncation=True, padding="max_length", max_length=8192, return_tensors="pt")
    targets = tokenizer(target_text, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

    return {
        "input_ids": inputs["input_ids"][0],
        "attention_mask": inputs["attention_mask"][0],
        "labels_tokenized": targets["input_ids"][0]  # Tokenized target for training
    }


In [ ]:
processed_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names, batched=False)

# Set PyTorch format for DataLoader, but include `labels` as a string for ROUGE
processed_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels_tokenized"])


In [ ]:
from torch.utils.data import DataLoader

# Define batch size
batch_size = 16

# Create a DataLoader for batching
train_loader = DataLoader(processed_dataset["train"], batch_size=batch_size, shuffle=True)


scores = []

sample_limit = 128

count = 0
# Evaluation loop with ROUGE scoring
for batch in train_loader:
    count += 1
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels_tokenized = batch["labels_tokenized"]


    # Decode inputs to strings
    input_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in input_ids]
    target_texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in labels_tokenized]

    
    # Generate model responses
    responses = []
    for input_text in input_texts:
        try:
            response = qa(input_text)
            responses.append(response.data)
        except Exception as e:
            responses.append("API_ERROR")
            print(f"API call failed: {e}")

    # Compute ROUGE scores
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    for target, response in zip(target_texts, responses): 
        scores.append(scorer.score(target, response))

    if sample_limit <= batch_size * count:
        break

In [ ]:
n = 10
input_str = input_texts[n]
response_text = responses[n] 
targ = target_texts[n]

print("input:", input_str)
print("\n\ntarget text:", targ)
print("\n\nresponce text:", response_text)

In [ ]:
print(len(input_texts))
print(len(target_texts))
print(len(responses))

In [ ]:
#n = 22
#n = 5
#n = 56
n = 9

input_str = input_texts[n]
response_text = responses[n] 
targ = target_texts[n]

print("input:", input_str)
print("\n\ntarget text:", targ)
print("\n\nresponce text:", response_text)

In [ ]:
for i in range(len(target_texts)):
    print(target_texts[i])

In [ ]:
for i in range(len(responses)):
    print(responses[i])

In [ ]:
suum = 0
for i in range(len(scores)):
    suum += float(scores[i]['rougeL'][2])
print(suum / len(scores))

In [ ]:
print(len(scores))

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Define two sentences
# sentence1 = "Subtract(12.393, 22.366), divide(#0, 22.366)"
# sentence2 = "subtract(22.366, 12.393), divide(#0, 12.393)"

sentence1 = "The capital of France is Paris."
sentence2 = "Paris is not the capital of France."


# Create a TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the sentences into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform([sentence1, sentence2])

# Calculate cosine similarity
similarity = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Print the similarity score
print(f"Cosine Similarity: {similarity[0][0]:.4f}")


In [ ]:
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Correct answer and LLM output
correct_answer = "The capital of France is Paris."
llm_output = "Paris is not the capital of France."

# Generate embeddings
correct_embedding = model.encode(correct_answer, convert_to_tensor=True)
llm_embedding = model.encode(llm_output, convert_to_tensor=True)

# Compute cosine similarity
similarity = util.cos_sim(correct_embedding, llm_embedding)

print(f"Cosine Similarity: {similarity.item():.4f}")


In [ ]:
!pip install sentence_transformers